# 1.2  LP Basics

# Farmer Jane problem

The solution of this problem in Julia is list below.
The mathematical modelling is shown in scanned picture and graphic solution.

In [1]:
using JuMP, Clp

m = Model()

@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

@objective(m, Max, 200*x1 + 300*x2)

@constraint(m, worker,  3*x1 + 2*x2 <= 100)

@constraint(m, fertilizer,  2*x1 + 4*x2 <= 120)

@constraint(m, area, x1 + x2 <= 45)
;

In [2]:
println("Time used by Clp:")

set_optimizer(m, Clp.Optimizer)

@time(optimize!(m))

println("Wheat area", value(x1), "acres.")
println("Corn area", value(x2), "acres.")
println("Total profit will be \$", objective_value(m))

Time used by Clp:
 19.712441 seconds (45.15 M allocations: 2.226 GiB, 8.72% gc time)
Wheat area19.99999999999999acres.
Corn area20.000000000000007acres.
Total profit will be $10000.0
Coin0506I Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Dual inf 500 (2)
Clp0006I 3  Obj 10000
Clp0000I Optimal - objective value 10000
Clp0032I Optimal objective 10000 - 3 iterations time 0.002


# Top Brass problem

# Base

In [3]:
using JuMP, Clp

m = Model()

@variable(m, ft >= 0)
@variable(m, st >= 0)
@variable(m, kt >= 0)

@objective(m, Max, 12*ft + 9*st + 10*kt)

@constraint(m, wood_con, 4ft + 2st + 3*kt <= 4800)

@constraint(m, plaque_con, ft + st + kt<= 1750)

# constraints on brass footballs, soccerballs available
@constraint(m, brass_football_con, ft <= 1000)
@constraint(m, brass_soccerball_con, st <= 1500)
@constraint(m, brass_karate_con, kt <= 750)
; 

In [4]:
println("Time to solve this model using Clp: ")

set_optimizer(m, Clp.Optimizer)

@time(optimize!(m))

println("Build ", value(ft), " football trophies.")
println("Build ", value(st), " soccer trophies.")
println("Build ", value(kt), " karate trophies.")
println("Total profit will be \$", objective_value(m))

Time to solve this model using Clp: 
  0.001266 seconds (1.73 k allocations: 114.391 KiB)
Build 650.0 football trophies.
Build 1100.0 soccer trophies.
Build 0.0 karate trophies.
Total profit will be $17700.0
Coin0506I Presolve 2 (-3) rows, 3 (0) columns and 6 (-3) elements
Clp0006I 0  Obj -0 Dual inf 30.999997 (3)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


# Modular

In [7]:
trophy_types = [:football, :soccer, :karate] # these are the possible trophy types

wood_req = Dict(:football => 4, :soccer => 2, :karate =>3) # how much wood each trophy type will use

plaque_req = Dict(:football => 1, :soccer => 1, :karate => 1) # how many plaques each trophy type will use

profit = Dict( :football => 12, :soccer => 9, :karate => 10) # profit produced by each trophy type

wood_avail = 4800
plaques_avail = 1750
football_avail = 1000
soccer_avail = 1500
karate_avail = 750;

In [8]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, trophy[trophy_types] >= 0)

@objective(m, Max, sum(profit[i] * trophy[i] for i in trophy_types) )   

@constraint(m, sum(wood_req[i] * trophy[i] for i in trophy_types) <= wood_avail)
@constraint(m, sum(plaque_req[i] * trophy[i] for i in trophy_types) <= plaques_avail)
@constraint(m, trophy[:football] <= football_avail)
@constraint(m, trophy[:soccer] <= soccer_avail)
@constraint(m, trophy[:karate] <= karate_avail)

status = optimize!(m)

println(value.(trophy))
# note the output is quite ugly. we'll see how to make it look nicer soon.
println("Total profit will be \$", objective_value(m))
println("We will use ", value(sum(wood_req[i] * trophy[i] for i in trophy_types) ), " board feet of wood")
println("We will use ", value(sum(plaque_req[i] * trophy[i] for i in trophy_types) ), " plaques")

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer, :karate]
And data, a 3-element Array{Float64,1}:
  650.0
 1100.0
    0.0
Total profit will be $17700.0
We will use 4800.0 board feet of wood
We will use 1750.0 plaques
Coin0506I Presolve 2 (-3) rows, 3 (0) columns and 6 (-3) elements
Clp0006I 0  Obj -0 Dual inf 30.999997 (3)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


# Compact

In [9]:
trophy_types = [:football, :soccer, :karate]

resources = [:wood, :plaques, :brass_football, :brass_soccer, :brass_karate]

profit = Dict( zip(trophy_types, [12, 9, 10]))

resource_avail = Dict( zip(resources, [4800, 1750, 1000, 1500, 750] ) );

using NamedArrays

trophy_resource_matrix = [4 1 1 0 0
                          2 1 0 1 0
                          3 1 0 0 1]

trophy_resource_NA = NamedArray(trophy_resource_matrix, (trophy_types, resources), ("type","resource"))

# check out the output to see how NamedArrays are structured:

3×5 Named Array{Int64,2}
type ╲ resource │           :wood  …    :brass_karate
────────────────┼────────────────────────────────────
:football       │               4  …                0
:soccer         │               2                   0
:karate         │               3  …                1

In [10]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, trophy[trophy_types] >= 0)

@expression(m, tot_profit, sum(profit[i] * trophy[i] for i in trophy_types) )

@constraint(m, constr[i in resources], sum(trophy_resource_NA[t, i] * trophy[t] for t in trophy_types) <= resource_avail[i] )

@objective(m, Max, tot_profit)

optimize!(m)

println(value.(trophy))
println("Total profit will be \$", objective_value(m))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer, :karate]
And data, a 3-element Array{Float64,1}:
  650.0
 1100.0
    0.0
Total profit will be $17700.0
Coin0506I Presolve 2 (-3) rows, 3 (0) columns and 6 (-3) elements
Clp0006I 0  Obj -0 Dual inf 30.999997 (3)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


## From above program modification, obviously from Base form to Compact form, the program becomes easier to be modified because of the variable separation.

# Stigler Diet Problem

In [ ]:
using DataFrames, CSV, NamedArrays

df=CSV.read("stigler.csv",header=true, delim=','); the name#s of the DataFrame (header) are the nutrients
nutrients=propertynames(df)[2:end]# create a list of foods from the diet array
foods=convert(Array,df[2:end,1])# turn dataframe into Array# create a dictionary of the min requirement of each nutrient
min_daily_req=Dict(zip(nutrients,df[1,2:end]))